In [1]:
import numpy as np
import cv2

In [2]:
# 1. Generating Artificial Data

# Intrinsic matrices for two cameras
K1 = np.array([
    [800, 0, 640],
    [0, 800, 360],
    [0, 0, 1]
])

K2 = np.array([
    [805, 0, 642],
    [0, 805, 362],
    [0, 0, 1]
])

# Generate random 3D points as ground truth
num_points = 10
X_3D = np.hstack((np.random.randint(0, 50, (num_points, 1)),
                  np.random.randint(0, 50, (num_points, 1)),
                  np.random.randint(50, 100, (num_points, 1))))

# Camera 1 is at the origin of the global coordinate system
P1 = K1 @ np.hstack((np.eye(3), np.zeros((3, 1))))

# Define a relative rotation and translation for Camera 2
R = cv2.Rodrigues(np.array([0.1, 0.2, 0.3]))[0]
t = np.array([[10], [5], [2]])

P2 = K2 @ np.hstack((R, t))

# Convert 3D points to homogeneous coordinates for projection
X_3D_homogeneous = np.hstack([X_3D, np.ones((num_points, 1))])

# Project the 3D points to the images
points1_homogeneous = P1 @ X_3D_homogeneous.T
points1 = (points1_homogeneous[:2, :] / points1_homogeneous[2, :]).T

points2_homogeneous = P2 @ X_3D_homogeneous.T
points2 = (points2_homogeneous[:2, :] / points2_homogeneous[2, :]).T


In [3]:
# 2. Stereo Triangulation

# Compute the fundamental matrix
F, _ = cv2.findFundamentalMat(points1, points2, cv2.FM_LMEDS)

# Compute the essential matrix
E = K1.T @ F @ K2

# Recover pose from essential matrix
_, R_recovered, t_recovered, _ = cv2.recoverPose(E, points1, points2, K1)

# Projection matrix for the second camera
P2_recovered = K2 @ np.hstack((R_recovered, t_recovered))

# Triangulate points
points_4D_hom = cv2.triangulatePoints(P1, P2_recovered, points1.T, points2.T)
points_3D_recovered = (points_4D_hom[:3, :] / points_4D_hom[3, :]).T


In [4]:
# 3. Comparison

# Reprojection errors
reprojected_points1_homogeneous = P1 @ points_4D_hom
reprojected_points1 = (reprojected_points1_homogeneous[:2, :] / reprojected_points1_homogeneous[2, :]).T

reprojected_points2_homogeneous = P2_recovered @ points_4D_hom
reprojected_points2 = (reprojected_points2_homogeneous[:2, :] / reprojected_points2_homogeneous[2, :]).T

error1 = np.sqrt(np.mean(np.sum((points1 - reprojected_points1)**2, axis=1)))
error2 = np.sqrt(np.mean(np.sum((points2 - reprojected_points2)**2, axis=1)))

total_reprojection_error = (error1 + error2) / 2

print(f"Mean Reprojection Error: {total_reprojection_error:.2f}")

Mean Reprojection Error: 36.83


In [5]:
points1

array([[ 680.4040404 ,  424.64646465],
       [ 968.57142857,  902.85714286],
       [ 972.30769231,  889.23076923],
       [ 964.05063291,  400.50632911],
       [ 765.        ,  393.33333333],
       [1081.02564103,  708.71794872],
       [ 805.85365854,  360.        ],
       [1078.0952381 ,  750.47619048],
       [ 734.91525424,  753.22033898],
       [ 789.15254237,  536.27118644]])

In [6]:
reprojected_points1

array([[ 672.68646622,  385.40131807],
       [ 963.15256848,  920.64998105],
       [ 967.89755815,  904.41112974],
       [ 956.10605879,  350.73067245],
       [ 755.68520688,  349.28264132],
       [1075.09628907,  745.99125821],
       [ 794.74363435,  312.51512109],
       [1070.12319838,  791.82707771],
       [ 740.39467541,  705.0385989 ],
       [ 787.49018005,  504.02377769]])

In [7]:
points2

array([[ 913.04627042,  422.24429564],
       [1099.0289878 ,  984.0527521 ],
       [1090.29611894,  967.41024174],
       [1246.64082398,  505.16842119],
       [1013.20191231,  420.72176412],
       [1256.81249708,  847.76240245],
       [1082.15872858,  408.63715254],
       [1230.4234415 ,  881.74392534],
       [ 911.40881731,  760.44706274],
       [1036.44749763,  588.71507418]])

In [8]:
reprojected_points2

array([[ 905.68366727,  464.23228812],
       [1099.24870234,  970.91308019],
       [1090.53336074,  956.17363606],
       [1240.97283633,  540.76002137],
       [1005.79038032,  462.4025262 ],
       [1259.38230793,  824.44341054],
       [1074.38161657,  450.65034875],
       [1232.9797116 ,  855.53296907],
       [ 910.17363863,  809.5615487 ],
       [1032.77861134,  617.97990418]])